# Trade Finance Automation with Document AI

## Use Case Overview

**Business Problem:** Oil & Gas refiners process hundreds of trade documents:
- **Letters of Credit (LCs)**: Payment guarantees for crude imports
- **Bills of Lading**: Shipping documents
- **Invoices**: Commercial invoices from suppliers
- **Certificates**: Quality, origin, insurance certificates

**Manual Processing Challenges:**
- 60-70% of LCs have discrepancies on first presentation
- Average 3-5 days to process a document set
- $150-500 per discrepancy handling cost

**AI Applications:**
1. OCR for document digitization
2. Named Entity Recognition (NER) for field extraction
3. Cross-document validation
4. Discrepancy prediction and auto-correction

---

## Learning Objectives

1. Simulate trade finance documents
2. Build OCR/extraction pipeline (simulated)
3. Create document validation engine
4. Implement discrepancy detection
5. Build processing dashboard

In [ ]:
# Imports
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
import warnings
warnings.filterwarnings('ignore')

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sys
sys.path.append('..')
from src.treasury_sim.generators import generate_trade_documents, set_seed

print("✅ Setup complete!")

## 1. Generate Trade Documents

### Thinking Trace 🧠

> **What documents are in a typical LC presentation?**
> 
> 1. **Letter of Credit**: The bank's payment guarantee
> 2. **Commercial Invoice**: Details of goods and pricing
> 3. **Bill of Lading**: Proof of shipment
> 4. **Certificate of Origin**: Country of origin
> 5. **Quality Certificate**: Product specifications
> 6. **Insurance Certificate**: Cargo insurance
>
> All documents must be **consistent** - any mismatch is a "discrepancy"

In [ ]:
set_seed(42)

# Generate trade documents
documents = generate_trade_documents(n_transactions=100, seed=42)

print(f"📊 Generated {len(documents):,} documents")
print(f"\nDocument Types: {documents['document_type'].value_counts().to_dict()}")

In [ ]:
# Preview documents
documents.head(10)

## 2. Simulated OCR & Document Extraction

### Thinking Trace 🧠

> **How does Document AI work?**
> 
> 1. **OCR Layer**: Converts scanned documents to text
>    - Azure Form Recognizer, AWS Textract, Google Document AI
> 
> 2. **NER/Extraction Layer**: Identifies key fields
>    - Amount, date, beneficiary, goods description
> 
> 3. **Confidence Scores**: Each extraction has confidence
>    - High confidence (>0.95): Auto-accept
>    - Medium (0.80-0.95): Review flag
>    - Low (<0.80): Manual review required

In [ ]:
class DocumentExtractor:
    """
    Simulates OCR and field extraction from trade documents.
    In production, this would use Azure Form Recognizer or similar.
    """
    
    def __init__(self):
        self.extraction_rules = {
            'LC': ['lc_number', 'amount', 'currency', 'beneficiary', 'applicant', 
                   'expiry_date', 'latest_shipment_date', 'goods_description'],
            'INVOICE': ['invoice_number', 'amount', 'currency', 'seller', 'buyer',
                        'goods_description', 'quantity', 'unit_price'],
            'BILL_OF_LADING': ['bl_number', 'vessel', 'port_of_loading', 'port_of_discharge',
                               'shipper', 'consignee', 'goods_description', 'quantity'],
            'CERTIFICATE_OF_ORIGIN': ['cert_number', 'country_of_origin', 'exporter',
                                       'goods_description', 'quantity'],
            'QUALITY_CERTIFICATE': ['cert_number', 'product_name', 'api_gravity', 
                                     'sulfur_content', 'inspection_date'],
            'INSURANCE_CERTIFICATE': ['policy_number', 'insured_amount', 'coverage_type',
                                       'voyage_from', 'voyage_to']
        }
    
    def extract(self, document_row):
        """
        Simulate extraction from a document.
        Returns extracted fields with confidence scores.
        """
        doc_type = document_row['document_type']
        fields = self.extraction_rules.get(doc_type, [])
        
        # Simulate extraction with varying confidence
        extracted = {
            'document_id': document_row['document_id'],
            'document_type': doc_type,
            'transaction_id': document_row['transaction_id'],
            'fields': {}
        }
        
        base_confidence = document_row.get('ocr_confidence', 0.95)
        
        for field in fields:
            # Simulate field-level confidence (varies by field type)
            if field in ['amount', 'currency', 'lc_number', 'bl_number']:
                # Structured fields - higher confidence
                confidence = min(base_confidence + np.random.uniform(0, 0.05), 1.0)
            elif field in ['goods_description', 'beneficiary', 'consignee']:
                # Text fields - lower confidence
                confidence = base_confidence - np.random.uniform(0, 0.15)
            else:
                confidence = base_confidence + np.random.uniform(-0.1, 0.05)
            
            # Get value from document (simulate extraction)
            if field == 'amount':
                value = document_row['amount']
            elif field == 'currency':
                value = document_row['currency']
            elif field == 'goods_description':
                value = document_row['goods_description']
            elif field == 'quantity':
                value = document_row['quantity']
            else:
                # Generate placeholder for other fields
                value = f"{field.upper()}_{document_row['transaction_id'][:8]}"
            
            extracted['fields'][field] = {
                'value': value,
                'confidence': round(confidence, 3)
            }
        
        return extracted

# Initialize extractor
extractor = DocumentExtractor()

# Extract from all documents
extractions = [extractor.extract(row) for _, row in documents.iterrows()]

print(f"✅ Extracted fields from {len(extractions)} documents")

In [ ]:
# Example extraction
import json

print("📄 SAMPLE EXTRACTION (Letter of Credit)")
print("=" * 60)

# Find an LC
lc_extraction = next(e for e in extractions if e['document_type'] == 'LC')
print(json.dumps(lc_extraction, indent=2, default=str))

In [ ]:
# Analyze extraction confidence
confidence_data = []
for ext in extractions:
    for field, data in ext['fields'].items():
        confidence_data.append({
            'document_type': ext['document_type'],
            'field': field,
            'confidence': data['confidence']
        })

confidence_df = pd.DataFrame(confidence_data)

# Confidence by document type
fig = px.box(confidence_df, x='document_type', y='confidence', color='document_type',
             title='Extraction Confidence by Document Type')
fig.add_hline(y=0.95, line_dash="dash", line_color="green", annotation_text="Auto-accept")
fig.add_hline(y=0.80, line_dash="dash", line_color="red", annotation_text="Manual review")
fig.update_layout(height=400)
fig.show()

## 3. Document Validation Engine

### Thinking Trace 🧠

> **What makes LC documents compliant?**
> 
> Key validation rules (UCP 600):
> 1. **Amount consistency**: Invoice ≤ LC amount
> 2. **Goods description**: Must match LC terms exactly
> 3. **Dates**: Documents within validity period
> 4. **Quantities**: B/L quantity matches invoice (±5% tolerance)
> 5. **Parties**: Names must match exactly
> 6. **Ports**: Must match LC requirements

In [ ]:
class DocumentValidator:
    """
    Validates trade documents against LC requirements.
    Detects discrepancies that could cause payment rejection.
    """
    
    def __init__(self):
        self.rules = [
            ('amount_match', self._check_amount_match),
            ('quantity_tolerance', self._check_quantity_tolerance),
            ('goods_description', self._check_goods_description),
            ('date_validity', self._check_date_validity),
            ('currency_match', self._check_currency_match)
        ]
        
    def _check_amount_match(self, docs_group):
        """Invoice amount must not exceed LC amount"""
        lc = docs_group[docs_group['document_type'] == 'LC']
        invoice = docs_group[docs_group['document_type'] == 'INVOICE']
        
        if lc.empty or invoice.empty:
            return None
            
        lc_amount = lc['amount'].iloc[0]
        inv_amount = invoice['amount'].iloc[0]
        
        if inv_amount > lc_amount:
            return {
                'rule': 'amount_match',
                'severity': 'HIGH',
                'message': f'Invoice amount ({inv_amount:,.0f}) exceeds LC amount ({lc_amount:,.0f})',
                'auto_fixable': False
            }
        return None
    
    def _check_quantity_tolerance(self, docs_group):
        """B/L quantity must be within ±5% of invoice"""
        invoice = docs_group[docs_group['document_type'] == 'INVOICE']
        bl = docs_group[docs_group['document_type'] == 'BILL_OF_LADING']
        
        if invoice.empty or bl.empty:
            return None
            
        inv_qty = invoice['quantity'].iloc[0]
        bl_qty = bl['quantity'].iloc[0]
        
        tolerance = abs(bl_qty - inv_qty) / inv_qty
        
        if tolerance > 0.05:  # 5% tolerance
            return {
                'rule': 'quantity_tolerance',
                'severity': 'MEDIUM',
                'message': f'Quantity mismatch: Invoice ({inv_qty:,.0f}) vs B/L ({bl_qty:,.0f}) = {tolerance:.1%}',
                'auto_fixable': True  # Can request amendment
            }
        return None
    
    def _check_goods_description(self, docs_group):
        """Goods description must match across documents"""
        descriptions = docs_group['goods_description'].unique()
        
        if len(descriptions) > 1:
            return {
                'rule': 'goods_description',
                'severity': 'HIGH',
                'message': f'Inconsistent goods descriptions: {list(descriptions)[:2]}',
                'auto_fixable': False
            }
        return None
    
    def _check_date_validity(self, docs_group):
        """Documents must be dated within validity period"""
        lc = docs_group[docs_group['document_type'] == 'LC']
        
        if lc.empty:
            return None
            
        # Check if any document is dated after LC expiry (simulated)
        # In real system, would compare actual dates
        if np.random.random() < 0.05:  # 5% chance of date issue
            return {
                'rule': 'date_validity',
                'severity': 'HIGH',
                'message': 'Document dated after LC expiry',
                'auto_fixable': False
            }
        return None
    
    def _check_currency_match(self, docs_group):
        """All documents must show same currency"""
        currencies = docs_group['currency'].dropna().unique()
        
        if len(currencies) > 1:
            return {
                'rule': 'currency_match',
                'severity': 'MEDIUM',
                'message': f'Currency mismatch: {list(currencies)}',
                'auto_fixable': True
            }
        return None
    
    def validate(self, docs_group):
        """Validate a set of documents for a single transaction"""
        discrepancies = []
        
        for rule_name, rule_func in self.rules:
            result = rule_func(docs_group)
            if result:
                discrepancies.append(result)
        
        return {
            'transaction_id': docs_group['transaction_id'].iloc[0],
            'n_documents': len(docs_group),
            'is_compliant': len(discrepancies) == 0,
            'discrepancies': discrepancies,
            'n_discrepancies': len(discrepancies),
            'severity_max': max([d['severity'] for d in discrepancies], default='NONE')
        }

# Initialize validator
validator = DocumentValidator()

# Validate all transactions
validation_results = []
for txn_id in documents['transaction_id'].unique():
    txn_docs = documents[documents['transaction_id'] == txn_id]
    result = validator.validate(txn_docs)
    validation_results.append(result)

validation_df = pd.DataFrame(validation_results)

print("📊 VALIDATION SUMMARY")
print("=" * 50)
print(f"Total Transactions: {len(validation_df)}")
print(f"Compliant: {validation_df['is_compliant'].sum()} ({validation_df['is_compliant'].mean()*100:.1f}%)")
print(f"With Discrepancies: {(~validation_df['is_compliant']).sum()}")
print(f"\nDiscrepancy Distribution:")
print(validation_df['n_discrepancies'].value_counts().sort_index())

In [ ]:
# Analyze discrepancies
all_discrepancies = []
for result in validation_results:
    for disc in result['discrepancies']:
        disc['transaction_id'] = result['transaction_id']
        all_discrepancies.append(disc)

if all_discrepancies:
    disc_df = pd.DataFrame(all_discrepancies)
    
    # Discrepancy breakdown
    fig = make_subplots(rows=1, cols=2,
        subplot_titles=('Discrepancies by Rule', 'Discrepancies by Severity'),
        specs=[[{"type": "pie"}, {"type": "pie"}]])
    
    # By rule
    rule_counts = disc_df['rule'].value_counts()
    fig.add_trace(go.Pie(labels=rule_counts.index, values=rule_counts.values), row=1, col=1)
    
    # By severity
    sev_counts = disc_df['severity'].value_counts()
    colors = {'HIGH': 'red', 'MEDIUM': 'orange', 'LOW': 'yellow'}
    fig.add_trace(go.Pie(labels=sev_counts.index, values=sev_counts.values,
                         marker_colors=[colors.get(s, 'gray') for s in sev_counts.index]), 
                  row=1, col=2)
    
    fig.update_layout(height=350, title_text='Discrepancy Analysis')
    fig.show()
else:
    print("✅ No discrepancies found!")

## 4. AI-Powered Discrepancy Resolution

### Thinking Trace 🧠

> **How can AI help resolve discrepancies?**
> 
> 1. **Classification**: Categorize discrepancy type
> 2. **Resolution Recommendation**: Suggest corrective action
> 3. **Auto-fix**: Apply corrections for minor issues
> 4. **Escalation**: Route complex issues to specialists
> 5. **Learning**: Track resolution patterns for improvement

In [ ]:
class DiscrepancyResolver:
    """
    AI-powered discrepancy resolution assistant.
    Recommends actions and tracks resolution patterns.
    """
    
    def __init__(self):
        self.resolution_patterns = {
            'amount_match': {
                'action': 'REQUEST_AMENDMENT',
                'template': 'Request LC amendment to increase amount to {required_amount}',
                'avg_resolution_days': 3,
                'cost_estimate': 350
            },
            'quantity_tolerance': {
                'action': 'NEGOTIATE_TOLERANCE',
                'template': 'Request bank acceptance under +/- 5% tolerance clause',
                'avg_resolution_days': 1,
                'cost_estimate': 150
            },
            'goods_description': {
                'action': 'REISSUE_DOCUMENT',
                'template': 'Request supplier to reissue {document_type} with corrected description',
                'avg_resolution_days': 4,
                'cost_estimate': 250
            },
            'date_validity': {
                'action': 'REQUEST_LC_EXTENSION',
                'template': 'Request LC extension or accept under reserve',
                'avg_resolution_days': 5,
                'cost_estimate': 500
            },
            'currency_match': {
                'action': 'CORRECT_DOCUMENT',
                'template': 'Request correction of currency to {correct_currency}',
                'avg_resolution_days': 2,
                'cost_estimate': 100
            }
        }
    
    def recommend_resolution(self, discrepancy):
        """Recommend resolution for a discrepancy"""
        rule = discrepancy['rule']
        pattern = self.resolution_patterns.get(rule, {
            'action': 'MANUAL_REVIEW',
            'template': 'Escalate to trade finance specialist',
            'avg_resolution_days': 7,
            'cost_estimate': 500
        })
        
        return {
            'discrepancy': discrepancy,
            'recommended_action': pattern['action'],
            'resolution_template': pattern['template'],
            'estimated_days': pattern['avg_resolution_days'],
            'estimated_cost': pattern['cost_estimate'],
            'auto_fixable': discrepancy.get('auto_fixable', False)
        }
    
    def calculate_impact(self, discrepancies):
        """Calculate business impact of all discrepancies"""
        total_cost = 0
        total_days = 0
        
        resolutions = []
        for disc in discrepancies:
            resolution = self.recommend_resolution(disc)
            resolutions.append(resolution)
            total_cost += resolution['estimated_cost']
            total_days = max(total_days, resolution['estimated_days'])
        
        return {
            'resolutions': resolutions,
            'total_estimated_cost': total_cost,
            'max_resolution_days': total_days,
            'n_auto_fixable': sum(1 for r in resolutions if r['auto_fixable'])
        }

# Initialize resolver
resolver = DiscrepancyResolver()

# Analyze resolution impact
if all_discrepancies:
    impact = resolver.calculate_impact(all_discrepancies)
    
    print("📊 DISCREPANCY RESOLUTION ANALYSIS")
    print("=" * 60)
    print(f"Total Discrepancies: {len(all_discrepancies)}")
    print(f"Auto-fixable: {impact['n_auto_fixable']}")
    print(f"\n💰 Estimated Resolution Cost: ${impact['total_estimated_cost']:,.0f}")
    print(f"⏱️  Max Resolution Time: {impact['max_resolution_days']} days")
    
    print("\n📋 RESOLUTION RECOMMENDATIONS:")
    print("-" * 60)
    for res in impact['resolutions'][:5]:  # Show first 5
        print(f"\n  Rule: {res['discrepancy']['rule']}")
        print(f"  Action: {res['recommended_action']}")
        print(f"  Cost: ${res['estimated_cost']} | Days: {res['estimated_days']}")

## 5. Processing Time Analytics

### Thinking Trace 🧠

> **How does AI reduce processing time?**
> 
> | Process Step | Manual | With AI |
> |-------------|--------|--------|
> | Document Receipt | 1 day | Minutes |
> | OCR/Digitization | 2-4 hours | 2-3 minutes |
> | Field Extraction | 1-2 hours | Seconds |
> | Cross-validation | 2-4 hours | Seconds |
> | Discrepancy ID | 1-2 hours | Instant |
> | **Total** | **1-2 days** | **<1 hour** |

In [ ]:
# Simulate processing time comparison
n_transactions = 100

processing_comparison = pd.DataFrame({
    'Step': ['Document Receipt', 'OCR/Digitization', 'Field Extraction', 
             'Cross-validation', 'Discrepancy Detection', 'Resolution Routing'],
    'Manual (hours)': [8, 3, 2, 3, 1.5, 1],
    'AI-Assisted (hours)': [0.5, 0.05, 0.01, 0.01, 0.01, 0.02]
})

processing_comparison['Time Savings (%)'] = (
    (processing_comparison['Manual (hours)'] - processing_comparison['AI-Assisted (hours)']) / 
    processing_comparison['Manual (hours)'] * 100
).round(1)

print("📊 PROCESSING TIME COMPARISON (per transaction)")
print("=" * 70)
print(processing_comparison.to_string(index=False))
print(f"\n⏱️  TOTAL Manual: {processing_comparison['Manual (hours)'].sum():.1f} hours")
print(f"⚡ TOTAL AI-Assisted: {processing_comparison['AI-Assisted (hours)'].sum():.2f} hours")
print(f"💰 Time Savings: {(1 - processing_comparison['AI-Assisted (hours)'].sum()/processing_comparison['Manual (hours)'].sum())*100:.1f}%")

In [ ]:
# Visualize processing comparison
fig = go.Figure()

fig.add_trace(go.Bar(
    name='Manual',
    x=processing_comparison['Step'],
    y=processing_comparison['Manual (hours)'],
    marker_color='indianred'
))

fig.add_trace(go.Bar(
    name='AI-Assisted',
    x=processing_comparison['Step'],
    y=processing_comparison['AI-Assisted (hours)'],
    marker_color='seagreen'
))

fig.update_layout(
    title='Trade Finance Processing Time: Manual vs AI-Assisted',
    yaxis_title='Hours per Transaction',
    barmode='group',
    height=400
)
fig.show()

## 6. Executive Dashboard

In [ ]:
# Create executive dashboard
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=(
        'Document Volume by Type', 'Compliance Rate', 'Processing Pipeline',
        'Discrepancy Trend', 'Cost Savings', 'SLA Performance'
    ),
    specs=[[{"type": "pie"}, {"type": "indicator"}, {"type": "funnel"}],
           [{"type": "scatter"}, {"type": "bar"}, {"type": "indicator"}]]
)

# 1. Document volume by type
doc_counts = documents['document_type'].value_counts()
fig.add_trace(go.Pie(labels=doc_counts.index, values=doc_counts.values, hole=0.4), row=1, col=1)

# 2. Compliance rate gauge
compliance_rate = validation_df['is_compliant'].mean() * 100
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=compliance_rate,
    title={'text': "First-Pass Compliance"},
    gauge={'axis': {'range': [0, 100]},
           'bar': {'color': "green" if compliance_rate > 50 else "red"},
           'threshold': {'line': {'color': "black", 'width': 4}, 'value': 50}}
), row=1, col=2)

# 3. Processing funnel
fig.add_trace(go.Funnel(
    y=['Received', 'Digitized', 'Extracted', 'Validated', 'Approved'],
    x=[100, 100, 98, 95, int(compliance_rate)],
    textinfo="value+percent initial"
), row=1, col=3)

# 4. Discrepancy trend (simulated)
dates = pd.date_range(end=datetime.now(), periods=30, freq='D')
disc_trend = np.random.poisson(5, 30) + np.linspace(8, 3, 30)  # Declining trend
fig.add_trace(go.Scatter(x=dates, y=disc_trend, mode='lines+markers', name='Discrepancies'), row=2, col=1)

# 5. Cost savings
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
savings = [15000, 22000, 28000, 35000, 42000, 48000]
fig.add_trace(go.Bar(x=months, y=savings, marker_color='green'), row=2, col=2)

# 6. SLA gauge
sla_performance = 94.5
fig.add_trace(go.Indicator(
    mode="gauge+number+delta",
    value=sla_performance,
    title={'text': "SLA Met %"},
    delta={'reference': 95},
    gauge={'axis': {'range': [80, 100]},
           'bar': {'color': "darkblue"},
           'steps': [{'range': [80, 90], 'color': 'lightgray'},
                     {'range': [90, 95], 'color': 'gray'}]}
), row=2, col=3)

fig.update_layout(height=600, title_text='Trade Finance Operations Dashboard', showlegend=False)
fig.show()

## 7. Key Takeaways

### AI Technologies for Trade Finance

| Technology | Application | Benefit |
|-----------|-------------|--------|
| **OCR** | Document digitization | 99%+ accuracy on typed text |
| **NER** | Field extraction | Automated data capture |
| **Rules Engine** | Compliance checking | Consistent validation |
| **ML Classification** | Discrepancy prediction | Proactive resolution |
| **LLM** | Resolution recommendations | Expert-level guidance |

### Business Impact

| Metric | Before AI | After AI | Improvement |
|--------|-----------|----------|-------------|
| Processing time | 2-3 days | <1 hour | **97%** |
| First-pass compliance | 35% | 65% | **86%** |
| Cost per document | $50 | $12 | **76%** |
| Discrepancy detection | Hours | Seconds | **99%** |

### Implementation Considerations

⚠️ **Important Notes:**
- OCR accuracy depends on document quality
- Human review required for edge cases
- Integration with bank systems needed
- Regulatory compliance (UCP 600) must be maintained

---

*Author: Ozgur Guler (ozgur.guler1@gmail.com)*